In [7]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU detected:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU detected.")


CUDA available: True
GPU detected: NVIDIA GeForce RTX 2080 Super with Max-Q Design


# Imports

In [16]:
import os
import pandas as pd
import torch
import gc
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from torch.nn.functional import softmax


# Data Prep

In [5]:
df = pd.read_csv("consolidated.csv", sep=";")

# check solidity exist, remove "nan" value
def has_source(fp_sol):
    sol_path = os.path.join(r"C:\Users\dafel\OneDrive - The Pennsylvania State University\Y4S2\DS 440\source", f"{fp_sol}.sol")
    return os.path.isfile(sol_path)

df = df[df["fp_sol"].apply(has_source)]

# transfer property_holds to label： "t" -> 1， "f" -> 0
df = df[df['property_holds'].isin(['t', 'f'])]
df['label'] = df['property_holds'].map({'t': 1, 'f': 0})

# read Solidity Source code, sol as txt
def read_source(fp_sol):
    sol_path = os.path.join(r"C:\Users\dafel\OneDrive - The Pennsylvania State University\Y4S2\DS 440\source", f"{fp_sol}.sol")
    try:
        with open(sol_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"file {sol_path} missing, continue")
        return ""

df['code'] = df['fp_sol'].apply(read_source)

# filter "nan" out，select code and label
data = df.dropna(subset=['code', 'label'])
data = data[data['code'] != ""]
solidity_data = []
for _, row in tqdm(data.iterrows(), total=len(data), desc="Loading Solidity files"):
    fp_sol = row["fp_sol"]
    sol_path = os.path.join("cgt-main", "source", f"{fp_sol}.sol")
    try:
        with open(sol_path, "r", encoding="utf-8") as f:
            code = f.read()
        solidity_data.append({
            "contract_name": row["contractname"],
            "code": code,
            "bug_type": row["property"],
            "swc_id": row["swc"],
            "dasp_id": row["dasp"]
        })
    except FileNotFoundError:
        pass
        
# JSON file saved
df_solidity = pd.DataFrame(solidity_data)
df_solidity.to_json("aaaaa.json", indent=4)

# train-validate separate
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['code'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions)
    }

# define PyTorch Dataset class
class SolidityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        # auto cut code lenth
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
# Define weighted loss inside a custom Trainer
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Manually set class weights here
        weight = torch.tensor([1.0, 2.5]).to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


Loading Solidity files: 100%|██████████| 19456/19456 [00:02<00:00, 8579.04it/s]


# CodeBERT

In [ ]:
# load CodeBERT tokenizer and model form（microsoft/codebert-base）
model_name = "microsoft/codebert-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# !!!!!!!!!!!!!!!!!!!!!!! need further explore
train_dataset = SolidityDataset(train_texts, train_labels, tokenizer)
val_dataset = SolidityDataset(val_texts, val_labels, tokenizer)

# Trainer define and tune
training_args = TrainingArguments(
    output_dir="./codebert-finetuned",  
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# TRAINING
trainer.train()

predictions = trainer.predict(val_dataset)

# Show result
eval_result = trainer.evaluate()
print("Evaluation results:", eval_result)

Loading Solidity files: 100%|██████████| 19456/19456 [00:02<00:00, 9161.62it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\dafel\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.699400,0.680210,0.709404
1000,0.660100,0.642834,0.677030
1500,0.653300,0.637949,0.734070
2000,0.642900,0.649253,0.743063
2500,0.640200,0.640382,0.721223


: 

In [ ]:
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

# Generate classification report
report = classification_report(true_labels, preds)
print(report)

              precision    recall  f1-score   support

           0       0.74      0.98      0.85      2825
           1       0.73      0.11      0.19      1067

    accuracy                           0.74      3892
   macro avg       0.74      0.55      0.52      3892
weighted avg       0.74      0.74      0.67      3892



# GraphCodeBert

In [ ]:

# Load tokenizer and model
model_name_gcb = "microsoft/graphcodebert-base"
tokenizer_gcb = RobertaTokenizer.from_pretrained(model_name_gcb)
model_gcb = RobertaForSequenceClassification.from_pretrained(model_name_gcb, num_labels=2)

# Dataset
train_dataset_gcb = SolidityDataset(train_texts, train_labels, tokenizer_gcb)
val_dataset_gcb = SolidityDataset(val_texts, val_labels, tokenizer_gcb)

# TrainingArguments
training_args_gcb = TrainingArguments(
    output_dir="./graphcodebert-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs_gcb',
    logging_steps=100,
    save_total_limit=2,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True
)


# Trainer
trainer_gcb = WeightedTrainer(
    model=model_gcb,
    args=training_args_gcb,
    train_dataset=train_dataset_gcb,
    eval_dataset=val_dataset_gcb,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_gcb.to(device)
trainer_gcb.train()
predictions_gcb = trainer_gcb.predict(val_dataset_gcb)
eval_result_gcb = trainer_gcb.evaluate()
print("GraphCodeBERT Eval:", eval_result_gcb)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# CodeT5+

In [20]:
# Clear CUDA cache
torch.cuda.empty_cache()

# Collect unused Python objects
gc.collect()

# Load tokenizer and model
# model_name_c5p = "Salesforce/codet5p-770m"
model_name_c5p = "Salesforce/codet5-base"
tokenizer_c5p = AutoTokenizer.from_pretrained(model_name_c5p)
model_c5p = AutoModelForSequenceClassification.from_pretrained(model_name_c5p, num_labels=2)

# Dataset
train_dataset_c5p = SolidityDataset(train_texts, train_labels, tokenizer_c5p)
val_dataset_c5p = SolidityDataset(val_texts, val_labels, tokenizer_c5p)

# TrainingArguments
training_args_c5p = TrainingArguments(
    output_dir="./codet5p-finetuned",
    num_train_epochs=3,
    fp16=True,
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs_c5p',
    logging_steps=100,
    save_total_limit=2,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True
)

# Trainer
trainer_c5p = WeightedTrainer(
    model=model_c5p,
    args=training_args_c5p,
    train_dataset=train_dataset_c5p,
    eval_dataset=val_dataset_c5p,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_c5p.to(device)
trainer_c5p.train()
predictions_c5p = trainer_c5p.predict(val_dataset_c5p)
eval_result_c5p = trainer_c5p.evaluate()
print("CodeT5+ Eval:", eval_result_c5p)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 